# 主要駅・都心駅へのアクセスを調査するエージェント

このノートブックでは、LangChain と LangGraph を使って複雑な Agent Workflow を構築します。

## 構築するワークフロー

駅から主要駅へのアクセスを調査して注目点をピックアップするエージェント

1. **近隣主要駅（自動選定）**
   - 駅周辺の主要な駅へのアクセスを抽出
2. **都内主要駅(新宿 / 渋谷 / 池袋 / 東京 / 品川 / 新橋 / 上野)**
   - 都心の主要な駅へのアクセスを抽出
3. **アクセスが良い主要駅**
   - 1.2.の結果を総合評価して注目ポイントを作成


In [1]:
# 必要なライブラリのインポート
import os
import json
from typing import List, Dict, Any, Annotated, Sequence
from dotenv import load_dotenv

# LangChain imports
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

# 環境変数読み込み
load_dotenv()

print("📦 ライブラリインポート完了")

# API キー確認
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY:
    print("✅ OpenAI APIキーが設定されています")
else:
    print("⚠️ OpenAI APIキーが設定されていません")
    print("💡 OpenAI APIキーの設定方法:")
    print("   1. https://platform.openai.com でアカウント作成")
    print("   2. API Keysセクションで新しいキーを作成")
    print("   3. .envファイルにOPENAI_API_KEY=your_key_hereを追記")

📦 ライブラリインポート完了
✅ OpenAI APIキーが設定されています


In [10]:
# State定義 - Agent間でやり取りするデータ構造  
from pydantic import BaseModel, Field, ConfigDict
from langgraph.graph import MessagesState


# Pydantic BaseModelでStationクラスを定義
class Station(BaseModel):
    model_config = ConfigDict(from_attributes=True)
    
    name: str = Field(..., description="駅名")
    duration_minutes: int = Field(..., description="最短所要時間（分）")
    route: str = Field(..., description="経路")
    num_transfers: int = Field(..., description="乗換回数")


# 選択理由付きのStationクラス
class StationChoice(BaseModel):
    model_config = ConfigDict(from_attributes=True)
    
    name: str = Field(..., description="駅名")
    duration_minutes: int = Field(..., description="最短所要時間（分）")
    route: str = Field(..., description="経路")
    num_transfers: int = Field(..., description="乗換回数")
    choice_reason: str = Field(..., description="選択理由")
    
class ResponseSchema(BaseModel):
    model_config = ConfigDict(from_attributes=True)
    
    nearby_stations: List[Station] = Field(..., description="駅近隣主要駅へのアクセス情報")
    major_stations: List[Station] = Field(..., description="都心主要駅へのアクセス情報")
    highlight_stations: List[StationChoice] = Field(..., description="駅近隣主要駅と都心主要駅のアクセス情報の注目ポイントリスト")


# MessagesStateを継承して独立したノード処理に対応したState定義
class AgentState(MessagesState):
    # 入力パラメータ
    station_name: Annotated[str, "基準となる最寄り駅名"]
    
    # 各ノードの処理結果（独立した処理ステップ）
    nearby_stations: Annotated[List[Station], "近隣主要駅のStation情報リスト"]
    major_stations: Annotated[List[Station], "都心主要駅のStation情報リスト"]  
    best_access_stations: Annotated[List[StationChoice], "アクセス良好駅の選択結果"]
    
    # 処理状態管理
    current_step: Annotated[str, "現在の処理ステップ"]
    processing_errors: Annotated[List[str], "処理中に発生したエラーのリスト"]

In [25]:
station_name = "天王町駅"
num_highlight_stations = 5
prompt = f"""あなたは、鉄道アクセス情報の専門リサーチャーです。
これから指定する「{station_name}」についてアクセス情報のみを調査し、
特に「アクセスが良い都内主要駅」を強調できるように、以下の要件に従って出力してください。

---

【調査対象1：近隣主要駅（自動選定）】
以下の基準で、{station_name}から10〜15分以内で行ける“近隣主要駅”を3〜5駅、自動的に選んでください。
- 乗降客数が多い、または乗換ターミナル機能があること

各駅について以下を出力してください。
- 駅名:
- 最短所要時間:
- 経路（使用路線・乗換駅）:
- 乗換回数:

---

【調査対象2：都内主要駅】
以下の駅について、アクセス情報を調査してください。
対象：新宿 / 渋谷 / 池袋 / 東京 / 品川 / 新橋 / 上野

各駅について以下を出力：
- 駅名:
- 最短所要時間:
- 経路:
- 乗換回数:

---

【調査対象3：“アクセスが良い主要駅”の抽出】
上記で調査した主要駅の中から、
以下の基準で「特にアクセスが良い駅」を{num_highlight_stations}駅まで抽出し、理由を添えて強調してください。

■ 抽出基準
- 所要時間が短い
- 経路がシンプル（混雑路線でない、方向が直線的など）
- 3駅のバリエーションがある（方向性、路線の多様性、 近隣駅/都内駅から少なくとも1つずつ選択）

■ 強調出力フォーマット
【アクセスが良い主要駅（3駅まで）】
- 駅名:
- 最短所要時間:
- 乗換回数:
- 選ばれた理由（簡潔に）:

---

【重要】
- 情報はすべて2025年時点の検索に基づく事実のみ。
- 推測・曖昧表現は禁止。
- 乗換経路は「◯◯線 → ◯◯線」のように具体的に記載。

---

【調査対象駅】
{station_name}
"""

In [26]:
llm = ChatOpenAI(model="gpt-5-search-api", temperature=0.0)
llm = llm.with_structured_output(ResponseSchema)
tmp = llm.invoke([("user", prompt)])

In [27]:
tmp.model_dump()

{'nearby_stations': [{'name': '横浜駅',
   'duration_minutes': 5,
   'route': '相鉄本線 各駅停車',
   'num_transfers': 0},
  {'name': '西横浜駅',
   'duration_minutes': 3,
   'route': '相鉄本線 各駅停車',
   'num_transfers': 0},
  {'name': '保土ケ谷駅',
   'duration_minutes': 15,
   'route': '相鉄本線 → 横須賀線',
   'num_transfers': 1}],
 'major_stations': [{'name': '新宿駅',
   'duration_minutes': 50,
   'route': '相鉄本線 → 東急東横線 → 東京メトロ副都心線 / 乗換複数（相鉄線特急～東急直通）',
   'num_transfers': 2},
  {'name': '渋谷駅',
   'duration_minutes': 40,
   'route': '相鉄本線 → 東急東横線直通',
   'num_transfers': 1},
  {'name': '池袋駅',
   'duration_minutes': 55,
   'route': '相鉄本線 → 東急東横線 → 東京メトロ副都心線',
   'num_transfers': 1},
  {'name': '東京駅',
   'duration_minutes': 45,
   'route': '相鉄本線 → 横浜駅でJR東海道線または横須賀線',
   'num_transfers': 1},
  {'name': '品川駅',
   'duration_minutes': 35,
   'route': '相鉄本線 → JR横須賀線・東海道線（横浜で乗換）',
   'num_transfers': 1},
  {'name': '新橋駅',
   'duration_minutes': 50,
   'route': '相鉄本線 → 横浜 → JR東海道線・京浜東北線・山手線',
   'num_transfers': 1},
  {'name'